# **Neurons for Visions**
## **Designing the Neural Network**
First, we'll look at the design of the neural network in *Figure 2-5*
![Extending our pattern for a more complex example](fig2.5.png)

```python
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        ])
```

The first, `Flatten` isn't a layer of neurons, but an input layer specification. Our inputs are $28\times 28$ images, but we want them to be treated as a series of numeric values, like a gray boxes at the top of the *Figure 2-5*. `Flatten` takes that "square" value (a 2D array) and turns it into a line (a 1D array).

The next one, `Dense`, is a layer of neurons, and we're specifying that we want 128 of them. This is the middle layer shown in *Figure 2-5*. You'll often hear such layers described as $\textbf{\textit{hidden layers}}$. Layers that are between the inputs and the outputs aren't seen by a caller, so the term "hidden" is used to describe them. We are asking for $128$ neurons to have their internal parameters randomly initialized. More neurons means it will run more slowly, as it has to learn more parameters. More neurons could also lead to a network that is greate at recognizing the training data, but not so good at recognizing data that it hasn't previously seen (this is known as $\textit{overfitting}$). On the other hand, fewer neurons means that the model might not have sufficient parameters to learn.

It takes some experimentation over time to pick the right values. This process is typically called $\textbf{\textit{hyperparameter
tuning}}$. In machine learning, a hyperparameter is a value that is used to control the training, as opposed to the internal values of the neurons that get trained/learned, which are referred to as parameters.

There's also an $\textit{activation function}$ specified in that layer. The activation function is code that will execute on each neuron in the layer. TensorFlow supports a number of them, but a very common one in middle layers is `relu`, which stands for $\textit{rectified linear unit}$. It's a simple function that just returns a value if it's greater than $0$. In this case, we don't want negative values being passed to the next layer to potentially impact the summing function, so instead of writing a lot of `if-then` code, we can simply activate the layer with `relu`.

Finally, there's another `Dense` layer, which is the output layer. This has $10$ neurons, because we have $10$ classes. Each of these neurons will end up with a probability that the input pixels match that class, so our job is to determine which one has the highest values. We could loop through them to pick that value, but the `softmax` activation function does that for us.

Now when we train our neural network, the goal is that we can feed in a $28 \times 28$-pixel array and the neurons in the middle layer will have weights and bias ($m$ and $c$ values) that when combined will match those pixels to one of the $10$ output values.

In [1]:
import tensorflow as tf

data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images  = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7802 - loss: 0.6356
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8601 - loss: 0.3897
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 943us/step - accuracy: 0.8742 - loss: 0.3454
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 993us/step - accuracy: 0.8833 - loss: 0.3178
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 939us/step - accuracy: 0.8911 - loss: 0.2934


First is a handy shortcut for accessing the data:
```python
    data = tf.keras.datasets.fashion_mnist
```
`keras` has a number of built-in datasets that you can access with a single line of code like this. In this case you don't have to handle downloading the $70,000$ images-spliting them into training and test sets, and so on-all it takes is one line of code.

We can call its `load_data` method to return our training and test sets like this:
```python
    (training_images, training_labels), 
    (test_images, test_labels) = data.load_data()
```
Fashion MNIST is designed to have $60,000$ training images and $10,000$ test images. So the return from `data.load_data` will give you an array of $60,000 $ $28 \times 28$-pixel arrays called `training_images`, and an array of $60,000$ values (0-9) called `training_labels`. Similarly, the `test_images` will contain $10,000$ 28 $\times$ 28-pixel arrays, and the `test_labels` array will contain $10,000$ values between 0 and 9.


```python
    training_images  = training_images / 255.0
    test_images = test_images / 255.0
```
Python allows you to do an operation across the entire array with this notation. Recall that all of the pixels in our images are grayscale, with values between 0 and 255. Dividing by 255 thus ensures that every pixel is represented by a number between 0 and 1 instead. This process is called $\textbf{\textit{normalizing}}$ the image.

Normalization will improve performance. Often your network will not learn and will have massive errors when dealing with non normalized data.

Next we define the neural network that makes up our model, as discussed ealier:
```python
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
```
When we compile our model we specify the loss function and the optimizer as before:
```python
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
```
The loss function in this case is called $\textit{sparse categorical cross entropy}$, and it's one of the arsenal of loss functions that are built into TensorFlow. Again, choosing which loss function to use is an art in itself, and over time you'll learn which ones are best to use in which scenarios. Here we're picking a *category*. Our item of clothing will belong to $1$ of $10$ categories of clothing, and thus using a *categorical* loss function is the way to go. Sparse categorical cross entropy is a good choice

The same applies to choosing an optimizer. The `adam` optimizer is an evolution of the stochastic gradient descent (`sgd`) optimizers that has been shown to be faster and more efficient. As we're handling $60,000$ training images, any performance improvement we can get will be helpful.

Next, we'll train the network by fitting the training images to the training labels over five epochs:
```python
    model.fit(training_images, training_labels, epochs=5)
```


Finally, we can do something new-evaluate the model, using a single line of code. We have a set of $10,000$ images and lebels for testing, and we can pass them to the trained model to have it predict what it thinks each image is, compare that to its actual label, and sum up the result:
```python
    model.evaluate(test_images, test_labels)
```

## **Training the Neural Network**
```python
    58016/60000 [=====>.] - ETA: 0s - loss: 0.2941 - accuracy: 0.8907
    59552/60000 [=====>.] - ETA: 0s - loss: 0.2943 - accuracy: 0.8906
    60000/60000 [] - 2s 34us/sample - loss: 0.2940 - accuracy: 0.8906
```
Note that now it's now reporting accuracy. So in this case, using the training data, our model ended up with an accuracy of about $89\%$ after only five epochs.

But what about the test data? The result of `model.evaluate` on our test data will look something like this:
```python
    10000/1 [====] - 0s 30us/sample - loss: 0.2521 - accuracy: 0.8736
```
In this case the accuracy of the model was $87.36\%$, which isn't bad considering we only trained it for five epochs.

You're probably wondering why the accuracy is *lower* for the test data than it is for the training data. This is very commonly seen but it makes sense: the neural network only really knows how to match the inputs it has been trained on with the ouputs for those values. Our hope is that, given enough data, it will be able to generalize from the examples it has seen, "learning" what a shoe or a dress looks like. But there will always be examples of items that is hasn't seen that are sufficiently different from what it has to confuse it.

## **Exploring the Model Output**
Now that the model has been trained, and we have a good gage of its accuracy using the test set, let's explore it a little:

In [2]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step
[8.61319450e-07 6.27786623e-09 9.87816406e-07 1.45292930e-07
 8.74490797e-06 2.54593417e-02 4.99701719e-06 5.83301485e-02
 1.14903996e-04 9.16079819e-01]
9


The classification gives us back an array of values. These are the values of the $10$ output neurons. The label is the actual label for the item of clothing, in this case $9$. You'll see that some of the values are very small, and the last one is the largest by far. These are the probabilities that the image matches the label at that particular index. So, what the neural network is reporting is that there's a $91.4\%$ chance that the item of clothing at index $0$ is label $9$. We know that it's label $9$, so it got it right.

## **Training for Longer-Discovering Overfitting**
In this case, we trained for only five epochs. That is, we went through the entire training loop of having the neurons randomly initialized, checked against their labels, having that performance measured by the loss function, and then updated by the optimizer five times. And the result we got were pretty good: $89\%$ accuracy on the training set and $87\%$ on the test set.

Try to update it to train for $50$ epochs instead of $5$. In this case, we got these accuracy figures on the training set:
```python
    58112/60000 [==>.] - ETA: 0s - loss: 0.0983 - accuracy: 0.9627
    59520/60000 [==>.] - ETA: 0s - loss: 0.0987 - accuracy: 0.9627
    60000/60000 [====] - 2s 35us/sample - loss: 0.0986 - accuracy: 0.9627
```

This is particularly exciting because we're doing much better: $96.27\%$ accuracy. For the test set we reach $88.6\%$
```python
    [====] - 0s 30us/sample - loss: 0.3870 - accuracy: 0.8860
```

So, we got a big improvement on the training set, and a smaller one on the test set. This might suggest that training our network for much longer would lead to much better results—but that’s not always the case. The network is doing much better with the training data, but it’s not necessarily a better model. In fact, the divergence in the accuracy numbers shows that it has become overspecialized to the training data, a process often called $\textbf{\textit{overfitting}}$. As you build more neural networks this is something to watch out for, and as you go through this book you’ll learn a number of techniques to avoid it.

## **Stopping Training**
In each of the cases so far, we've hardcoded the number of epochs we're training for. While that works, we might want to train until we reach the desired accuracy instead of constantly trying different numbers of epochs and training and retraining until we get to our desired value. So, for example, if we want to train until the model is at $95\%$ accuracy on the training set, without knowing how many epochs that will take, how could we do that?

The easiest approach is to use a $\textbf{\textit{callback}}$ on the training.

In [1]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential({
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
})

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=50, callbacks=[callbacks])

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0070 - loss: 5.1278
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0076 - loss: 4.8765
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0090 - loss: 4.8466
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0123 - loss: 4.8284
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0133 - loss: 4.8071
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0144 - loss: 4.7896
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0146 - loss: 4.7797
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0130 - loss: 4.7661
Epoch 9/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0141 - loss: 4.7557
Epoch 10/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0155 - loss: 4.7466
Epoch 11/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.0130 - loss: 4.7443
Epoch 12/50
1875/1875 ━━━━━━━━

First, we created a new class called `myCallback`. This takes a `tf.keras.callbacks.Callback` as a parameter. In it, we define the `on_epoch_end` function, which will give us details about the logs for this epoch. In these logs is an accuracy value, so all we have to do is see if it is greater than $.95$; if it is, we can stop training by saying `self.model.stop_training = True`

Once we've specified this, we create a `callbacks` object to be an instance of the `myCallback` function.

Now check out the `model.fit` statement. You'll see that I've updated it to train for 50 epochs, and then added a `callbacks` parameter. To this, I pass the `callbacks` object.

When training, at the end of every epoch, the callback funcion will be called. So at the end of each epoch you'll check, and after about 34 epochs you'll ee that your training will end, because the training has hit $95\%$ accuracy:
```python
    56896/60000 [====>..] - ETA: 0s - loss: 0.1309 - accuracy: 0.9500
    58144/60000 [====>.] - ETA: 0s - loss: 0.1308 - accuracy: 0.9502
    59424/60000 [====>.] - ETA: 0s - loss: 0.1308 - accuracy: 0.9502
    Reached 95% accuracy so cancelling training!
```